In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
### local
#root_path = '../data/open'

### colab
root_path = '/content/drive/MyDrive/12조 파이널프로젝트/data'

In [3]:
import pandas as pd
df1 = pd.read_parquet(f'{root_path}/train/2.신용정보/201807_train_신용정보.parquet')

In [4]:
df2 = pd.read_parquet(f'{root_path}/train/2.신용정보/201808_train_신용정보.parquet')

In [5]:
df3 = pd.read_parquet(f'{root_path}/train/2.신용정보/201809_train_신용정보.parquet')

In [6]:
df4 = pd.read_parquet(f'{root_path}/train/2.신용정보/201810_train_신용정보.parquet')

In [7]:
df5 = pd.read_parquet(f'{root_path}/train/2.신용정보/201811_train_신용정보.parquet')

In [8]:
df6 = pd.read_parquet(f'{root_path}/train/2.신용정보/201812_train_신용정보.parquet')

In [9]:
# df1 ~ df6을 리스트에 담는다
month_list = [df1, df2, df3, df4, df5, df6]

# pd.concat으로 수직 결합 (행 기준)
month = pd.concat(month_list, ignore_index=True)

# 결과 확인
print(month.shape)
month.head()

(2400000, 42)


,기준년월,ID,최초한도금액,카드이용한도금액,CA한도금액,일시상환론한도금액,월상환론한도금액,CA이자율_할인전,CL이자율_할인전,RV일시불이자율_할인전,...,연체감액여부_R3M,한도심사요청건수,한도요청거절건수,한도심사요청후경과월,한도심사거절후경과월,시장단기연체여부_R6M,시장단기연체여부_R3M,시장연체상환여부_R6M,시장연체상환여부_R3M,rv최초시작후경과일
0,201807,TRAIN_000000,0,19354,7270,0,0,22.995207,18.254978,17.264967,...,0,0회,0,3,3,0,0,0,0,99999999
1,201807,TRAIN_000001,0,9996,5718,41996,90611,14.793821,14.834873,10.622446,...,0,0회,0,3,3,0,0,0,0,322
2,201807,TRAIN_000002,0,88193,35207,0,0,22.014276,17.875321,17.155829,...,0,0회,0,3,3,0,0,0,0,2378
3,201807,TRAIN_000003,0,19062,6531,0,0,22.998014,22.999453,19.293674,...,0,0회,0,3,3,0,0,0,0,99999999
4,201807,TRAIN_000004,0,177222,47149,48000,155020,14.661948,10.897410,10.654587,...,0,0회,0,3,3,0,0,0,0,99999999


In [10]:
month.duplicated(subset=['ID', '기준년월']).sum()

np.int64(0)

In [11]:
# 1. customer_train_cleaned.parquet 불러오기
customer = pd.read_parquet(f'{root_path}/cleaned_회원정보.parquet')

# 2. '고객ID'와 'Segment' 컬럼만 추출
df7 = customer[['ID', 'Segment']].copy()
df7

,ID,Segment
0,TRAIN_000000,D
1,TRAIN_000001,E
2,TRAIN_000002,C
3,TRAIN_000003,D
4,TRAIN_000004,E
...,...,...
2399995,TRAIN_399995,E
2399996,TRAIN_399996,D
2399997,TRAIN_399997,C
2399998,TRAIN_399998,E


In [12]:
# ID 기준으로 Segment 병합
month = month.merge(df7, on='ID', how='left')

#  결과 확인
print(month.shape)
print(month['Segment'].isnull().sum())  # Segment 누락값 있는지 확인
month.head()

(14400000, 43)
0


,기준년월,ID,최초한도금액,카드이용한도금액,CA한도금액,일시상환론한도금액,월상환론한도금액,CA이자율_할인전,CL이자율_할인전,RV일시불이자율_할인전,...,한도심사요청건수,한도요청거절건수,한도심사요청후경과월,한도심사거절후경과월,시장단기연체여부_R6M,시장단기연체여부_R3M,시장연체상환여부_R6M,시장연체상환여부_R3M,rv최초시작후경과일,Segment
0,201807,TRAIN_000000,0,19354,7270,0,0,22.995207,18.254978,17.264967,...,0회,0,3,3,0,0,0,0,99999999,D
1,201807,TRAIN_000000,0,19354,7270,0,0,22.995207,18.254978,17.264967,...,0회,0,3,3,0,0,0,0,99999999,D
2,201807,TRAIN_000000,0,19354,7270,0,0,22.995207,18.254978,17.264967,...,0회,0,3,3,0,0,0,0,99999999,D
3,201807,TRAIN_000000,0,19354,7270,0,0,22.995207,18.254978,17.264967,...,0회,0,3,3,0,0,0,0,99999999,D
4,201807,TRAIN_000000,0,19354,7270,0,0,22.995207,18.254978,17.264967,...,0회,0,3,3,0,0,0,0,99999999,D


In [13]:
# month DataFrame을 12조 파이널프로젝트 폴더에 저장
month.to_parquet(f'{root_path}/train_신용정보.parquet', index=False)

In [17]:
import pandas as pd

# 1. 데이터 불러오기
credit_with_seg = pd.read_parquet(f'{root_path}/train_신용정보.parquet')

##### 1. 결측치 처리
# 'RV신청일자' 변수 삭제
credit_with_seg = credit_with_seg.drop(columns=['RV신청일자'])

# 'RV전환가능여부' → 'RV전환불가능여부'
credit_with_seg['RV_전환가능여부_이진'] = (credit_with_seg['RV전환가능여부'] == 'N').astype(int)
credit_with_seg = credit_with_seg.drop(columns='RV전환가능여부')

##### 2. 자료형 변환
# 자발한도감액횟수_R12M: '0회' → 0, '1회' → 1, ...
credit_with_seg['자발한도감액횟수_R12M'] = (
    credit_with_seg['자발한도감액횟수_R12M']
    .str.replace('회', '', regex=False)
    .astype(int)
)

# ‘한도증액횟수_R12M’ → '한도증액_R12M_여부': '0회' → 0, '1회이상' → 1
credit_with_seg['한도증액_R12M_여부'] = credit_with_seg['한도증액횟수_R12M'].map({
    '0회': 0,
    '1회이상': 1
}).astype(int)
credit_with_seg.drop(columns=['한도증액횟수_R12M'], inplace=True)

# '카드론동의여부': 'Y' → 1, 'N' → 0
credit_with_seg['카드론동의여부'] = credit_with_seg['카드론동의여부'].map({'Y': 1, 'N': 0}).astype(int)

# '한도심사요청건수' → '한도심사요청여부': '0회' → 0, '1회이상' → 1
credit_with_seg['한도심사요청여부'] = credit_with_seg['한도심사요청건수'].map({
    '0회': 0,
    '1회이상': 1
}).astype(int)
credit_with_seg.drop(columns=['한도심사요청건수'], inplace=True)

##### 3. 상관관계 분석
# 모든 값이 동일한 변수 삭제
credit_with_seg = credit_with_seg.drop(columns=['시장연체상환여부_R3M'])

##### 4. 변수 별 확인 및 파생 변수 생성
credit_with_seg['RV실사용여부'] = (credit_with_seg['RV약정청구율'] > 0).astype(int)
credit_with_seg['강제한도감액횟수_2회이상여부'] = (credit_with_seg['강제한도감액횟수_R12M'] > 1).astype(int)
credit_with_seg['강제한도감액금액_R12M_3이상여부'] = (credit_with_seg['강제한도감액금액_R12M'] > 2).astype(int)
credit_with_seg['상향가능CA한도금액_1여부'] = (credit_with_seg['상향가능CA한도금액'] == 1).astype(int)

# 카드이용한도금액_A수준복합 파생변수 생성
a_limit = credit_with_seg[credit_with_seg['Segment'] == 'A']['카드이용한도금액_B1M']
q1, q3 = a_limit.quantile(0.25), a_limit.quantile(0.75)
iqr = q3 - q1
lower, upper = q1 - 1.5 * iqr, q3 + 1.5 * iqr
a_B1M = a_limit[(a_limit >= lower) & (a_limit <= upper)].min()

a_limit = credit_with_seg[credit_with_seg['Segment'] == 'A']['카드이용한도금액_B2M']
q1, q3 = a_limit.quantile(0.25), a_limit.quantile(0.75)
iqr = q3 - q1
lower, upper = q1 - 1.5 * iqr, q3 + 1.5 * iqr
a_B2M = a_limit[(a_limit >= lower) & (a_limit <= upper)].min()

def classify_dual_limit(row):
    b1 = row['카드이용한도금액_B1M'] >= a_B1M
    b2 = row['카드이용한도금액_B2M'] >= a_B2M
    return 2 if b1 and b2 else 1 if b1 or b2 else 0

credit_with_seg['카드이용한도금액_A수준복합'] = credit_with_seg.apply(classify_dual_limit, axis=1)

##### 5. 메모리 최적화
for col in credit_with_seg.select_dtypes(include='int64').columns:
    if credit_with_seg[col].max() < 2_147_483_647:
        credit_with_seg[col] = credit_with_seg[col].astype('int32')

for col in credit_with_seg.select_dtypes(include='float64').columns:
    credit_with_seg[col] = credit_with_seg[col].astype('float32')


# 최종 저장
credit_with_seg.to_parquet(f'{root_path}/cleaned_신용정보.parquet', index=False)
